# Test of readout code for LTC2324 ADC Chip

The overlay contains both the readout module for the ADC and a dummy module to generate simulated data.  The interface to the LTC2324 is through PMOD with the following pinout
- PMODA[0]: external trigger input, active HI
- PMODA[1]: cvt output, active LOW
- PMODA[2]: sck output
- PMODA[3]: clkout input
- PMODA[4]: sdo serial data input

The dummy ADC connections are on PMODB with pins corresponding to the same pins on PMODA
- PMODB[1]: dummy_cvt input
- PMODB[2]: dummy_sck input
- PMODB[3]: dummy_clkout output
- PMODB[4]: dummy_sdo serial data output

If PMODB pins 1-4 are jumpered to the same pins on on PMOD, then the module should read back the data written to the dummy_ADC register defined below.

In [1]:
# General imports
import ipywidgets as widgets
import numpy as np

In [2]:
# Because of a bug/feature of the Pynq library, this is necessary when loading updated
# versions of the same design.
from pynq import PL
PL.reset()

In [4]:
# load the overlay
from pynq import Overlay
pynq = Overlay("mani_readout.bit")
# help(pynq)

In [5]:
# Define the communication registers

# Interface to LTC2324 readout module
timing = pynq.timing   # timing control word
                       # timing[7:4] = number of clock cyccles for cvt low before first valid bit
                       # timing[3:0] = number of clock cycles for SCK_LO and SCK_HI
control = pynq.control # control word
                       # control[0] = arm
                       # control[1] = soft trigger
                       # control [6:2] - (not used)
                       # control [7] - polarity of external trigger 
                       #                 0-> active HI
                       #                 1-> active LO
state = pynq.state     # state of readount.  There will be data ready when state==4
# Data registers
data = [pynq.data1,pynq.data2,pynq.data3,pynq.data4]


# Interface to dummy data generation module
dummy_ADC = pynq.dummy_ADC  # Dummy data for the data simulation


# Utility routines

The following routines are used to convert and display ADC values

In [7]:
# Convert a 16 bit 2s complement value to a signed python integer
# Courtesy of ChatGPT
def twos_complement_16bit(value):
    if value & (1 << 15):  # check if the sign bit is set
        return value - (1 << 16)
    else:
        return value

In [8]:
# Convert 16 bit, 2s complement ADC readout to voltage
Vref = 2.    # range from -Vref to +Vref
def voltage(raw):
    return(twos_complement_16bit(raw)*Vref/2**15)

In [10]:
# Print out the raw ADC values
def print_raw():
    print("Raw ADC values...")
    for i in range(4):
        print("  Channel %d = 0x%X"%(i,data[i].read(0)))


In [11]:
# Print out the voltages
def print_voltages():
    print("ADC input voltages...")
    for i in range(4):
        print("  Channel %d = %.4f V"%(i,voltage(data[i].read(0))))


In [12]:
# This will return a string with the the voltages
def display_string():
    display = ""
    for i in range(4):
        display = display+" Chan%d=%.4fV"%(i,voltage(data[i].read(0)))
        if i<3:
            display = display+","
    return display
        

In [15]:
# This will create an ipywidgets text display to display the voltages
import ipywidgets as widgets
from ipywidgets import Text, Layout
from IPython.display import display

def create_voltage_display():
    voltage_field = widgets.Text(
        value=display_string(),
        description='ADC Values:',layout=Layout(width='50%')
    )
    # Return
    return voltage_field

# Generic code follows...

In [17]:
voltage_field=create_voltage_display()
display(voltage_field)

Text(value=' Chan0=0.0000V, Chan1=0.0000V, Chan2=0.0000V, Chan3=0.0000V', description='ADC Values:', layout=La…

In [19]:
voltage_field.value=display_string()

In [ ]:
# Initial setup
timing.write(0,0xff)     # 80 ns for everything
dummy_val = 0xAA55
dummy_ADC.write(0,dummy_val)  # dummy ADC data

In [ ]:
# Perform an arm, trigger, and readout
print("Expect State 4 (DONE), read state =%d"%state.read(0))       # should initially be in state 4 (DONE)

# Arm
control.write(0,1)  # arm
print("Armed.    Expect State 0 (READY), read state =%d"%state.read(0))       # should initially be in state 4 (DONE)

# Trigger
control.write(0,2)  # trigger
print("Triggered Expect State 4 (DONE), read state =%d"%state.read(0))       # should initially be in state 4 (DONE)

In [ ]:
# Print out the raw values
def print_raw():
    print("Raw ADC values...")
    for i in range(4):
        print("  Channel %d = 0x%X"%(i,data[i].read(0)))
